<a href="https://colab.research.google.com/github/Kalana99/190530H_ML_Mini_Project/blob/main/Layer_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np          # For mathematical calculations

from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt  # For plotting graphs
from pandas import Series        # To work on series
%matplotlib inline
import warnings                   # To ignore the warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('/content/drive/MyDrive/ML Mini Project/Data/L10/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML Mini Project/Data/L10/test.csv')
valid = pd.read_csv('/content/drive/MyDrive/ML Mini Project/Data/L10/valid.csv')

# Find columns with missing values and count how many missing values in each column
missing_columns = train.columns[train.isnull().any()]
missing_counts = train[missing_columns].isnull().sum()

# Print the columns with missing values and their corresponding missing value counts
print("shape of train: ", train.shape)
for column in missing_columns:
    print(f"Column '{column}' has {missing_counts[column]} missing values.")

shape of train:  (28520, 772)
Column 'label_2' has 480 missing values.


In [ ]:
L1 = "label_1" #Speaker ID
L2 = "label_2" #Speaker age
L3 = "label_3" #Speaker gender
L4 = "label_4" #Speaker accent
LABELS = [L1, L2, L3, L4,]
AGE_LABEL = L2
FEATURES = [f'feature_{i}' for i in range(1,769)]

In [ ]:
#OutLier analysis
data = train[0:5]
# Set the style of seaborn
sns.set(style="whitegrid")

# Create a box plot for each feature to visualize outliers
for col in data.columns:
    plt.figure(figsize=(8, 6))
    sns.boxplot(data[col])
    plt.title(f'Box Plot of {col}')
    plt.xlabel(col)
    # plt.show()

In [ ]:
#Heat Map

sns.heatmap(train, annot=True, cmap='coolwarm', center=0)

In [ ]:
train_df = train.copy()
test_df = test.copy()
valid_df = valid.copy()

train_df.head()

In [ ]:
train_df[LABELS + [FEATURES[i] for i in range(0,768)]].describe()

In [ ]:
train_df[L4].value_counts()

In [ ]:
train_df.info()

In [ ]:
test_df.head()

In [ ]:
from sklearn.preprocessing import RobustScaler # eliminate outliers

x_train = {}
x_valid = {}
x_test = {}

y_train = {}
y_valid = {}
y_test = {}

#create dictionaries for each label
for target_label in LABELS:
  tr_df = train_df[train_df['label_2'].notna()] if target_label == "label_2" else train_df
  vl_df = valid_df[valid_df['label_2'].notna()] if target_label == "label_2" else valid_df
  te_df = test_df

  scaler = RobustScaler()

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(tr_df.drop(LABELS, axis=1)), columns=FEATURES)
  y_train[target_label] = tr_df[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(vl_df.drop(LABELS, axis=1)), columns=FEATURES)
  y_valid  [target_label] = vl_df[target_label]

  x_test[target_label] = pd.DataFrame(scaler.transform(te_df.drop(['ID'], axis=1)), columns=FEATURES)
  # y_test[target_label] = te_df[target_label] <- need to predict

In [ ]:
x_train_df = x_train[L1].copy()
y_train_df = y_train[L1].copy()

x_valid_df = x_valid[L1].copy()
y_valid_df = y_valid[L1].copy()

x_test_df = x_test[L1].copy()
# y_test__df = y_test[L1].copy() <- need to predict

In [ ]:
from sklearn.utils import class_weight
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import confusion_matrix

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


In [ ]:
classifier = svm.SVC(kernel='linear')
classifier.fit(x_train_df, y_train_df)

# Accuarcy Check
y_pred = classifier.predict(x_valid_df)
# print("confusion_matrix:\n ",metrics.confusion_matrix(y_valid_df, y_pred))
print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_pred))
print("f1_score: ",f1_score(y_valid_df, y_pred, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
print("recall_score: ",metrics.recall_score(y_valid_df, y_pred, average='weighted'))

y_test_pred = classifier.predict(x_test_df)

### **FEATURE ENGINEERING**

In [ ]:
# @title 1. Feature selection using SelectKBest and f_classif
from sklearn.feature_selection import SelectKBest, f_classif, chi2
# from sklearn.neighbors import KNeighborsClassifier
#chi2 (Chi-square) cannot be used when datafarame has negative numbers

#Validation
selector = SelectKBest(f_classif, k=150)
x_new = selector.fit_transform(x_train_df, y_train_df)
x_valid_df_after_f_classif = selector.transform(x_valid_df)
print("shape: ", x_new.shape)


# #Testing
x_test_df_after_f_classif = selector.transform(x_test_df)

# # model train by Initialize kNN classifier
# k = 5  # Number of neighbors
# classifier = KNeighborsClassifier(n_neighbors=k)
# # Train the kNN classifier on the selected features
# classifier.fit(x_new, y_train_df)

# # Accuarcy Check
# y_pred = classifier.predict(x_valid_df_after_f_classif)
# # print("confusion_matrix: ",metrics.confusion_matrix(y_valid_df, y_pred))
# print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_pred))
# print("f1_score: ",f1_score(y_valid_df, y_pred, average='weighted')) #f-1 score
# print("precision_score: ",metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
# print("recall_score: ",metrics.recall_score(y_valid_df, y_pred, average='weighted'))


shape:  (28520, 150)


In [ ]:
# @title 2. Select Percentile for feature engineering
import pandas as pd
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score

#Validation
# Initialize SelectPercentile with f_classif scoring and percentile=10 (adjust as needed)
selector = SelectPercentile(score_func=f_classif, percentile=40)
# Fit and transform the data
x_new = selector.fit_transform(x_new, y_train_df)
x_valid_df_after_selectPercentile = selector.transform(x_valid_df_after_f_classif)
print("shape: ", x_new.shape)

#Testing
x_test_df_after_selectPercentile = selector.transform(x_test_df_after_f_classif)

# # model train by Initialize kNN classifier
# k = 5  # Number of neighbors
# classifier = KNeighborsClassifier(n_neighbors=k)
# # Train the kNN classifier on the selected features
# classifier.fit(x_new, y_train_df)

# # Accuarcy Check
# y_pred = classifier.predict(x_valid_df_after_selectPercentile)
# # print("confusion_matrix: ",metrics.confusion_matrix(y_valid_df, y_pred))
# print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_pred))
# print("f1_score: ",f1_score(y_valid_df, y_pred, average='weighted')) #f-1 score
# print("precision_score: ",metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
# print("recall_score: ",metrics.recall_score(y_valid_df, y_pred, average='weighted'))


shape:  (28520, 60)


In [ ]:
# @title 3. PCA for feature engineering
from sklearn.decomposition import PCA

#Validation
pca = PCA(n_components=0.98, svd_solver='full')
pca.fit(x_new)
x_train_trf = pd.DataFrame(pca.transform(x_new))
x_valid_trf_pca = pd.DataFrame(pca.transform(x_valid_df_after_selectPercentile))
print('Shape after PCA: ',x_train_trf.shape)

# # model train by Initialize kNN classifier
# k = 5  # Number of neighbors
# classifier = KNeighborsClassifier(n_neighbors=k)
# # Train the kNN classifier on the selected features
# classifier.fit(x_train_trf, y_train_df)

# # Accuarcy Check
# y_pred = classifier.predict(x_valid_trf_pca)
# # print("confusion_matrix:\n ",metrics.confusion_matrix(y_valid_df, y_pred))
# print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_pred))
# print("f1_score: ",f1_score(y_valid_df, y_pred, average='weighted')) #f-1 score
# print("precision_score: ",metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
# print("recall_score: ",metrics.recall_score(y_valid_df, y_pred, average='weighted'))


Shape after PCA:  (28520, 51)


In [ ]:
# @title 4. VarianceThreshold for feature engineering
from sklearn.feature_selection import VarianceThreshold

threshold = 0.55
selector = VarianceThreshold(threshold=threshold)
x_new = selector.fit_transform(x_train_df)
x_valid_df_after_VarianceThreshold = selector.transform(x_valid_df)
print("shape: ", x_new.shape)

# # model train
# clf = svm.SVC(kernel="linear")
# clf.fit( x_new, y_train_df)

# # Accuarcy Check
# y_pred = clf.predict(x_valid_df_after_VarianceThreshold)
# print(metrics.confusion_matrix(y_valid_df, y_pred))
# print(metrics.accuracy_score(y_valid_df, y_pred))
# print(metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
# print(metrics.recall_score(y_valid_df, y_pred, average='weighted'))

shape:  (28520, 610)


### **Model Evaluation**

In [ ]:
# @title 1. SVM
# model train
svm_clf = svm.SVC(kernel="linear")
svm_clf.fit( x_new, y_train_df)

# Accuarcy Check
y_pred = svm_clf.predict(x_valid_df_after_VarianceThreshold)
# print(metrics.confusion_matrix(y_valid_df, y_pred))
print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_pred))
print("f1_score: ",f1_score(y_valid_df, y_pred, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
print("recall_score: ",metrics.recall_score(y_valid_df, y_pred, average='weighted'))

accuracy_score:  0.9453333333333334
f1_score:  0.9464394754272543
precision_score:  0.9523410379094589
recall_score:  0.9453333333333334


In [ ]:
# @title 2. K-NN
# model train by Initialize kNN classifier
k = 5  # Number of neighbors
knn_clf = KNeighborsClassifier(n_neighbors=k)
# Train the kNN classifier on the selected features
knn_clf.fit(x_new, y_train_df)

# Accuarcy Check
y_pred = knn_clf.predict(x_valid_df_after_VarianceThreshold)
# print(metrics.confusion_matrix(y_valid_df, y_pred))
print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_pred))
print("f1_score: ",f1_score(y_valid_df, y_pred, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
print("recall_score: ",metrics.recall_score(y_valid_df, y_pred, average='weighted'))

accuracy_score:  0.8693333333333333
f1_score:  0.8710878796533734
precision_score:  0.8864625705638493
recall_score:  0.8693333333333333


In [ ]:
# @title 3. Random Forest
# model train by Initialize kNN classifier
n = 60  # Number of estimators
rf_clf = RandomForestClassifier(n_estimators=n)
# Train the kNN classifier on the selected features
rf_clf.fit(x_new, y_train_df)

# Accuarcy Check
y_pred = rf_clf.predict(x_valid_df_after_VarianceThreshold)
# print(metrics.confusion_matrix(y_valid_df, y_pred))
print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_pred))
print("f1_score: ",f1_score(y_valid_df, y_pred, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
print("recall_score: ",metrics.recall_score(y_valid_df, y_pred, average='weighted'))

accuracy_score:  0.8213333333333334
f1_score:  0.821335569393388
precision_score:  0.8333448753402314
recall_score:  0.8213333333333334


### **Hyper-parameter Tunning**

In [ ]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from scipy.stats import randint

clf = RandomForestClassifier(random_state=0)
np.random.seed(0)

param_distributions = {"max_depth": [3, None],
                        "min_samples_split": randint(2, 11)}
search = HalvingRandomSearchCV(clf, param_distributions,
                                resource='n_estimators',
                                max_resources=10,
                                random_state=0).fit(x_new, y_train_df)
search.best_params_

{'max_depth': None, 'min_samples_split': 10, 'n_estimators': 9}

In [ ]:
from sklearn.model_selection import GridSearchCV

svm_clf = svm.SVC()

param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf'],  # Kernel type
    'gamma': ['scale', 'auto', 0.1, 1],  # Kernel coefficient
}

grid_search = GridSearchCV(estimator=svm_clf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(x_new, y_train_df)

best_svm = grid_search.best_estimator_

In [ ]:
y_pred = search.predict(x_valid_df_after_VarianceThreshold)
# print(metrics.confusion_matrix(y_valid_df, y_pred))
print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_pred))
print("f1_score: ",f1_score(y_valid_df, y_pred, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
print("recall_score: ",metrics.recall_score(y_valid_df, y_pred, average='weighted'))

In [ ]:
y_pred = best_svm.predict(x_valid_df_after_VarianceThreshold)
# print(metrics.confusion_matrix(y_valid_df, y_pred))
print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_pred))
print("f1_score: ",f1_score(y_valid_df, y_pred, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(y_valid_df, y_pred, average='weighted' ))
print("recall_score: ",metrics.recall_score(y_valid_df, y_pred, average='weighted'))